★事前準備★

ご自分の「Googleドライブ」の「MyDrive」に 以下のフォルダ構成と

任意でGoogleColab上の「Stable-Diffusion_WEB-UI（Forge）」で使用したいデータを格納してください。

MyDrive

└StableDiffusion

 　└checkpoints　※使用したいモデルデータを格納

 　└controlnet　※ツール用モデルを格納

 　└embeddings　※EasyNegativeなどを格納

 　└lora　※使用したいLoRAデータを格納

 　└VAE　※使用したいVAEデータを格納

 　└outputs　※生成データのコピー用受け皿フォルダ（フォルダのみでOK）

 ※A1111で使っているStyleを流用したい場合は、

 マイドライブの「StableDiffusion」フォルダに「styles.csv」のファイルを保存しておいてください。

★このノートブックの使い方★

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～

①まずメニュー項目の「ランタイム」から
 → ランタイムの変更（いずれかのGPUタイプを選択）

②【01】の実行ボタン（▶）を押す、

（▶）の左に✅が付いたら次の（▶）を順番に押していく

※または「ランタイム」から「すべてのセルを実行」を実行する

③【03】の処理が進み、 https://************.gradio.live
と表示されたら、そのURLをクリック

④WEB-UIを停止するときは【03】の（▶）を押して処理をクルクルを停止

⑤再度WEB-UIを使用する際は、【03】の（▶）を押し③と同様

～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～


以下、「StableDiffusion_WebUI Forge環境」の起動手順です。

👇【01】Googleドライブのデータをこのノートブックへマウント（接続）

※Googleドライブにある、checkpoint、LoRA、VAE、Embedding、ControlNet用のツールモデル等を参照できるようにする

↓の（▶）を押し、出現するポップアップウィンドウから

　 Googleドライブに接続 → 「アカウントの選択」 → 「アクセスを許可」

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

👇【02】WEB-UIと各種_拡張機能のクローン（起動前にあらかじめ導入）※お好みで増減可能

In [ ]:
# Stable Diffusion Web-UI Forge のクローン（インストール）
!git clone https://github.com/lllyasviel/stable-diffusion-webui-forge.git
%cd /content/stable-diffusion-webui-forge

# 拡張機能リポジトリのURLリスト
extensions = [
    "https://github.com/chrisgoringe/Styles-Editor.git",
    "https://github.com/Bing-su/adetailer.git",
    "https://github.com/thomasasfk/sd-webui-aspect-ratio-helper.git",
    "https://github.com/Physton/sd-webui-prompt-all-in-one.git",
    "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111.git",
    "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git",
    "https://github.com/KohakuBlueleaf/z-tipo-extension.git",
    "https://github.com/lobehub/sd-webui-lobe-theme.git",
    "https://github.com/Gourieff/sd-webui-reactor-sfw.git",
    "https://github.com/zixaphir/Stable-Diffusion-Webui-Civitai-Helper.git",
]

# 拡張機能のクローン処理
%cd /content/stable-diffusion-webui-forge/extensions
for ext in extensions:
    !git clone {ext}
%cd /content/stable-diffusion-webui-forge

👇【03】WEB-UIの起動（各種大容量データはGoogleドライブを参照する設定）

 ※生成した画像は自動的に/content/drive/MyDrive/StableDiffusion/outputs へ保存されます

 ※起動オプション追記もお好みで可能です

★ ↓の（▶）で WEB-UIの「起動」「停止」「再起動」を操作します。



In [ ]:
# 出力ディレクトリを設定するスクリプトを作成
script_content = """
import os
from modules import shared

# Google Driveの出力ディレクトリを設定
shared.opts.data['outdir_txt2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-images'
shared.opts.data['outdir_txt2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/txt2img-grids'
shared.opts.data['outdir_img2img_samples'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-images'
shared.opts.data['outdir_img2img_grids'] = '/content/drive/MyDrive/StableDiffusion/outputs/img2img-grids'
shared.opts.data['outdir_extras_images'] = '/content/drive/MyDrive/StableDiffusion/outputs/extras-images'

print("出力ディレクトリの設定が完了しました。")
"""
# スクリプトを保存
script_path = "/content/stable-diffusion-webui-forge/scripts/set_output_dir.py"
with open(script_path, "w") as f:
    f.write(script_content)

#style.csvファイルコピー ※MyDrive/StableDiffusionに保存したファイルをコピーして、登録済みのStyleをForgeで使用可能にします
!cp "/content/drive/MyDrive/StableDiffusion/styles.csv" "/content/stable-diffusion-webui-forge"

# 設定スクリプトの実行とWEB-UIの起動
!python /content/stable-diffusion-webui-forge/scripts/set_output_dir.py
!python launch.py --share --no-half-vae --disable-nan-check \
  --ckpt-dir "/content/drive/MyDrive/StableDiffusion/checkpoints" \
  --embeddings-dir "/content/drive/MyDrive/StableDiffusion/embeddings" \
  --lora-dir "/content/drive/MyDrive/StableDiffusion/lora" \
  --vae-dir "/content/drive/MyDrive/StableDiffusion/vae" \
  --controlnet-dir "/content/drive/MyDrive/StableDiffusion/controlnet"